In [ ]:
%pip install pandas
%pip install gradio
%pip install folium
%pip install geopy

In [3]:
import pandas as pd
import gradio as gr
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic


# 데이터프레임 로드
APT = pd.read_csv(r"./data/aparts.csv")
APT = APT.dropna(subset=['좌표X', '좌표Y'])

# 교육시설 데이터 로드
EDU = pd.read_csv(r"./export/education/final_graded_school_locations.csv")
EDU = EDU.dropna(subset=['lat', 'lon'])

# 서울시의 구 목록
districts = APT['주소(시군구)'].unique().tolist()

# 선택한 구의 아파트 목록을 가져오는 함수
def get_apartments(selected_district):
    filtered_apartments = APT[APT['주소(시군구)'] == selected_district]
    apartment_names = filtered_apartments['k-아파트명'].unique().tolist()
    return gr.Dropdown(choices=apartment_names , label="아파트 선택", interactive=True)
 # set_choices 사용

# 선택된 아파트 주변의 교육시설과 아파트 위치를 지도에 표시하는 함수
def plot_apartment_and_schools(selected_district, selected_apartment):
    # 선택된 아파트 데이터 필터링
    filtered_apartments = APT[(APT['주소(시군구)'] == selected_district) & (APT['k-아파트명'] == selected_apartment)]
    if filtered_apartments.empty:
        return "아파트 정보를 찾을 수 없습니다."

    apartment_row = filtered_apartments.iloc[0]
    apartment_location = [apartment_row['좌표Y'], apartment_row['좌표X']]

    # 지도 초기화 (아파트 위치로 중심 설정)
    m = folium.Map(location=apartment_location, zoom_start=15)
    marker_cluster = MarkerCluster().add_to(m)

    # 아파트 마커 추가
    folium.Marker(
        location=apartment_location,
        popup=f"{selected_apartment} (아파트)",
        icon=folium.Icon(color='red')
    ).add_to(marker_cluster)

    # 아파트 주변 1km 이내 교육시설 필터링 및 추가
    nearby_schools = []
    for _, edu_row in EDU.iterrows():
        school_location = [edu_row['lat'], edu_row['lon']]
        distance = geodesic(apartment_location, school_location).km
        if distance < 3:
            nearby_schools.append({
                "name": edu_row['학교명'],
                "location": school_location,
                "distance": distance
            })

    for school in nearby_schools:
        folium.Marker(
            location=school['location'],
            popup=f"{school['name']} ({school['distance']:.2f} km)",
            icon=folium.Icon(color='blue')
        ).add_to(m)

    return m._repr_html_()

# Gradio 인터페이스 구성
with gr.Blocks() as iface:
    gr.Markdown("## 서울시 아파트 지도")
    
    # 구 선택 드롭다운
    district_dropdown = gr.Dropdown(choices=districts, label="구 선택")
    print(district_dropdown)
    apartment_dropdown = gr.Dropdown(choices=[], label="아파트 선택", interactive=True)

    # 지도 출력
    map_output = gr.HTML(label="서울시 아파트 지도")
    
    # 구 선택 시 아파트 목록 업데이트
    district_dropdown.change(
        fn=get_apartments,
        inputs=district_dropdown,
        outputs=apartment_dropdown
    )
    print(district_dropdown)
    # 아파트 선택 시 지도 업데이트
    apartment_dropdown.change(fn=plot_apartment_and_schools, inputs=[district_dropdown, apartment_dropdown], outputs=map_output)
    live = True
    # 설명 추가
    gr.Markdown("구를 선택한 후 아파트를 선택하면 해당 아파트와 주변 교육시설이 지도에 표시됩니다.")

# Gradio 인터페이스 실행
iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://9e9ce736372e6e6161.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
pip install --upgrade gradio

You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
